In [ ]:
import os
os.environ["DATASET_PERIOD"] = "50"
os.environ["PMODE"] = "pmode_0002" # same as jetson orin
os.environ["STREVAL_TRAIN"] = "1"

os.chdir("/root/shared/Anytime-Lidar/tools")

import _init_path
import datetime
import time
import json
import math
from pathlib import Path

import torch
import gc
import sys
import pickle
import numpy as np
from alive_progress import alive_bar

from eval_utils import eval_utils
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

import matplotlib.pyplot as plt
import res_pred_utils

def get_dataset(cfg):
    log_file = ('./tmp_results/log_eval_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=0)
    test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, batch_size=1,
        dist=False, workers=0, logger=logger, training=False
    )

    return logger, test_set, test_loader, sampler


def get_streaming_eval_samples(num_ds_elems : int, period_ms : int, output_times_ms : np.ndarray, pred_dicts_arr):
    # Streaming eval
    #Now do manual sampling base on time

    #times_ns should be output_times_ms
    sampled_objects = []
    for i in range(num_ds_elems):
        sample_time_ms = i*period_ms
        aft = output_times_ms > sample_time_ms
        if aft[0] == True:
            sampled_objects.append(None) # Should be an empty pred dicts arr
#             print('0', end=' ')
        elif aft[-1] == False:
            sampled_objects.append(pred_dicts_arr[-1])
#             print('-1', end=' ')
        else:
            sample_idx = np.argmax(aft) - 1
            sampled_objects.append(pred_dicts_arr[sample_idx])
#             print(sample_idx, end=' ')
#     print()

    return sampled_objects

def do_eval(sampled_objects, model, dump_eval_dict=True):
    #Convert them to openpcdet format
    dataset = model.dataset
    
    det_annos = []
    num_ds_elems = len(dataset)
    for i in range(num_ds_elems):
        data_dict = dataset.get_metadata_dict(i)
        for k, v in data_dict.items():
            data_dict[k] = [v] # make it a batch dict
        pred_dicts = sampled_objects[i]

        if pred_dicts is None:
            pred_dicts = [{
                'pred_boxes': torch.empty((0, 9)),
                'pred_scores': torch.empty(0),
                'pred_labels': torch.empty(0, dtype=torch.long)
            }]
        data_dict['final_box_dicts'] = pred_dicts
        det_annos += dataset.generate_prediction_dicts(
            data_dict, data_dict['final_box_dicts'], dataset.class_names, output_path=None
        )

    #nusc_annos = {} # not needed but keep it anyway
    result_str, result_dict = dataset.evaluation(
        det_annos, dataset.class_names,
        eval_metric=model.model_cfg.POST_PROCESSING.EVAL_METRIC,
        output_path='./tmp_results',
        boxes_in_global_coords=False,
    )

    print(result_str)
    if dump_eval_dict:
        eval_d = {
        'cfg': cfg,
        'det_annos': det_annos,
        'annos_in_glob_coords': False,
        'resolution': model.res_idx
        }
    
        eval_d['result_str'] = result_str
    
        with open(f'eval_data_res{model.res_idx}.pkl', 'wb') as f:
            pickle.dump(eval_d, f)
    return result_str

In [ ]:
import importlib
importlib.reload(res_pred_utils)

def calc_tail_ms(cur_time_point_ms, data_period_ms):
    return cur_time_point_ms - math.floor(cur_time_point_ms / data_period_ms) * data_period_ms

def get_egovel():
    

def do_streval_and_collect_data(resolution_idx, dump_eval_dict=True, sched_period_ms=2000):
    print('***********************')
    print(f'***RESOLUTION INDEX {resolution_idx}**')
    print('***********************')

    cfg_file = "./cfgs/nuscenes_models/pillar01_015_02_024_03_valor.yaml"
    cfg_from_yaml_file(cfg_file, cfg)
    
    set_cfgs = ['MODEL.METHOD', '0', 'MODEL.DEADLINE_SEC', '100.0', 'MODEL.DENSE_HEAD.NAME', 'CenterHeadInf',
                'OPTIMIZATION.BATCH_SIZE_PER_GPU', '1']
    cfg_from_list(set_cfgs, cfg)
    logger, test_set, test_loader, sampler = get_dataset(cfg)
    print(f'Loaded dataset with {len(test_set)} samples')
    
    ckpt_file="../output/nuscenes_models/pillar01_015_02_024_03_valor/default/ckpt/checkpoint_epoch_30.pth"
    
    model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
    model.load_params_from_file(filename=ckpt_file, logger=logger, to_cpu=False)
    # model.pre_hook_handle.remove()
    # model.post_hook_handle.remove()
    model.eval() # should be run with @torch.no_grad
    model.cuda()

    data_period_ms = 50
    num_samples = len(test_set)
    
    cur_sample_idx = 0
    sim_cur_time_ms = 0.
    last_exec_time_ms = 100.
    target_sched_time_ms = 0.
    pred_dicts_arr = []
    output_times_ms = []
    processed_inds = set()
    
    model.calibrate()
    do_res_sched = (resolution_idx == -1)
    model.res_idx = 0 if do_res_sched else resolution_idx

    resolution_stats = [0] * model.num_res
    with alive_bar(num_samples, force_tty=True, max_cols=160, manual=True) as bar:
        while cur_sample_idx < num_samples:
            lbd = model.latest_batch_dict # save bef its modified
            with torch.no_grad():
                #batch_dict = model.dataset.collate_batch([data_dict])
                #load_data_to_gpu(batch_dict)
                pred_dicts, ret_dict = model([cur_sample_idx])
    
            # Predict the execution time as if the DNN were to be executed on target platform
            batch_dict = model.latest_batch_dict
            num_points = batch_dict['points'].size(0)
            num_voxels = np.array([batch_dict['bb3d_num_voxels']])
            xlen = batch_dict['x_lims'][1] - batch_dict['x_lims'][0]
            last_exec_time_ms = model.calibrators[model.res_idx].pred_exec_time_ms(
               num_points, num_voxels, xlen)

            sim_cur_time_ms += last_exec_time_ms

            if do_res_sched and sim_cur_time_ms >= target_sched_time_ms:
                sample_tkn = batch_dict['metadata'][0]['token']
                if lbd is not None and not batch_dict['scene_reset']:
                    # time diff can be more than 50ms, but its ok
                    prev_sample_tkn = lbd['metadata'][0]['token']
                    egovel = res_pred_utils.get_2d_egovel(
                            model.token_to_ts[prev_sample_tkn],
                            model.token_to_pose[prev_sample_tkn],
                            model.token_to_ts[sample_tkn],
                            model.token_to_pose[sample_tkn])
                    
                else: # assume its zero
                    egovel = np.zeros(2)

                #NOTE Time prediction does not work all the time, 
                # res_exec_times_sec = []
                # for ridx in range(model.num_res):
                #     if ridx == model.res_idx:
                #         res_exec_times_sec.append(last_exec_time_ms)
                #     else:
                #         # WARNING!!!!! num voxels and xlen change depending on resolution!
                #         res_exec_times_sec.append(model.calibrators[ridx].pred_exec_time_ms(
                #                 num_points, num_voxels, xlen))
                # print(res_exec_times_sec)
                res_exec_times_sec = np.array([0.247, 0.147, 0.107, 0.091, 0.077]) # mean
                ratio = (last_exec_time_ms/1000.) / res_exec_times_sec[model.res_idx]
                chosen_res = res_pred_utils.pick_best_resolution(res_exec_times_sec * ratio,
                                                                 egovel, pred_dicts[0])
                resolution_stats[chosen_res] += 1
                model.res_idx = chosen_res
                #NOTE I need to consider the sched time as well and add to sim cur time ms
                target_sched_time_ms += sched_period_ms
            
            pred_dicts_arr.append(pred_dicts)
            output_times_ms.append(sim_cur_time_ms)
            processed_inds.add(cur_sample_idx)
    
            #Dynamic scheduling
            cur_tail = calc_tail_ms(sim_cur_time_ms, data_period_ms)
            pred_finish_time = sim_cur_time_ms + last_exec_time_ms #NOTE I can also use mean exec time
            next_tail = calc_tail_ms(pred_finish_time, data_period_ms)
            if next_tail < cur_tail:
                # Sleep, extra 1 ms is added to make sure sleet time is enough
                sim_cur_time_ms += data_period_ms - cur_tail + 1
    
            cur_sample_idx = int(sim_cur_time_ms / data_period_ms)
            if cur_sample_idx in processed_inds:
                print(f'ERROR, trying to process already processed sample {cur_sample_idx}')
    
            bar(cur_sample_idx / num_samples)
    
    output_times_ms = np.array(output_times_ms)
    model.print_time_stats()
    print('Resolution selection stats:')
    print(resolution_stats)
    
    #We have:
    # pred_dicts_arr
    # output_times_ms
    # processed_inds

    os.environ["RESOLUTION_IDX"] = str(model.res_idx)
    sampled_objects = get_streaming_eval_samples(len(test_set), data_period_ms, output_times_ms, pred_dicts_arr)
    print(f'Sampled {len(sampled_objects)} objects')
    result_str = do_eval(sampled_objects, model, dump_eval_dict)
    return resolution_stats, result_str

# from IPython.display import clear_output
# os.environ["FINE_GRAINED_EVAL"] = "1"
# for res_idx in range(5):
#     do_streval_and_collect_data(res_idx, True)

results = []
os.environ["FINE_GRAINED_EVAL"] = "0"
resolution_stats, result_str = do_streval_and_collect_data(-1, False)
results.append((resolution_stats, result_str))

with open("output.txt", "w") as f:
    for period, resolution_stats, result_str in results:
        f.write(f'{resolution_stats}\n')
        f.write(result_str)
        f.write('\n')